In [ ]:
#Import packages
import numpy as np
import matplotlib.pyplot as plt
import sys
import time as tm
import datetime as dt
import scipy.optimize as opt

In [ ]:
#Define functions

#Fit function
def fitFunc(x, A, B, C, D, E):
    return A/(np.exp(B*x + C) + D) + E

#Fit function if ideal fit fails -- fifth-order polynomial
def fitFunc_arb(x, A, B, C, D, E):
    return A + B*x + C*x**2 + D*x**3 + E*x**4

#Function to fit the data
def fit(x, y):
    try:
        popt = opt.curve_fit(fitFunc, x, y)[0]
        yfit = fitFunc(x, *popt)
        yend = fitFunc(100, *popt) #Temperature after 100 hours
    except:
        popt = opt.curve_fit(fitFunc_arb, x, y)[0]
        print popt
        yfit = fitFunc_arb(x, *popt)
        yend = fitFunc_arb(100, *popt) #Temperature after 100 hours
    return yfit, yend

#Convert Kikusui logger file time to binary time
def tConv(time):
    arr = time.rstrip(']').lstrip('[').split()
    year, month, day = arr[0].split('-')
    hour, minute, sec = arr[1].split(':')
    dt = datetime.datetime(int(year), int(month), int(day), int(hour), int(minute), int(sec))
    btime = tm.mktime(dt.timetuple())
    return int(btime)

#Generate start and end times for voltage steps
def genTimes(karr):
    start = 'initial current' #Flag for start of voltage step
    finish = 'final current' #Flat for end of voltage step
    power = 'final power'
    stimes = []
    ftimes = []
    powers = []
    for i in range(len(karr[0])):
        if start in karr[1][i]:
            stimes.append(tConv(karr[0][i]))
        elif finish in karr[1][i]:
            ftimes.append(tConv(karr[0][i]))
        elif power in karr[1][i]:
            powers.append(float(karr[1][i].split()[-2]))
        else:
            continue
    return stimes, ftimes, powers